## Exploring the Data

In [1]:
import os
import polars as pl
from transform_data import *

Let's read the data corresponding to a single state ('AC' in this case) to explore it.

In [2]:
csv_files_dir = '../data/csv_files'
file = 'AC.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv(file_path, separator = ',')

Let's check the different types of 'ClassInfraFisica' present.

In [3]:
df.group_by('ClassInfraFisica').count()

ClassInfraFisica,count
str,u32
"""Rooftop""",129
"""Ran Sharing""",39
"""Greenfield""",2532
null,6108
"""Indoor""",7
"""Streetlevel """,9
"""Streetlevel""",15


Let's fix the typo:

In [4]:
df = replace_values(df, 'ClassInfraFisica', 'Streetlevel ', 'Streetlevel')

In [5]:
df.group_by('ClassInfraFisica').agg(pl.col('ClassInfraFisica').count().alias('Count'))

ClassInfraFisica,Count
str,u32
"""Greenfield""",2532
"""Ran Sharing""",39
"""Indoor""",7
"""Streetlevel""",24
null,6108
"""Rooftop""",129


Now, we have to check whether each station (indicated by the column 'NumEstacao') corresponds to a single type of 'ClassInfraFisica'; which is the expected behavior (assumption).

In [6]:
count_unique_values = df.group_by('NumEstacao').agg(pl.col('ClassInfraFisica').n_unique().alias('unique_count'))
instances_with_different_values = count_unique_values.filter(count_unique_values['unique_count'] > 1)
print(instances_with_different_values)

shape: (162, 2)
┌────────────┬──────────────┐
│ NumEstacao ┆ unique_count │
│ ---        ┆ ---          │
│ i64        ┆ u32          │
╞════════════╪══════════════╡
│ 1006947148 ┆ 2            │
│ 686898176  ┆ 2            │
│ 1002285060 ┆ 2            │
│ 691336580  ┆ 2            │
│ …          ┆ …            │
│ 1009442667 ┆ 2            │
│ 684498219  ┆ 2            │
│ 684782715  ┆ 2            │
│ 1002900295 ┆ 2            │
└────────────┴──────────────┘


We found that we can actually get two different 'ClassInfraFisica' for any individual station. But if we check further, we'll see that it's actually pairing types with 'null' values, which does not compromise the assumption.

In [7]:
df.filter(df['NumEstacao'] == 699785804)[['NumEstacao', 'ClassInfraFisica']]

NumEstacao,ClassInfraFisica
i64,str
699785804,null
699785804,null
699785804,null
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""


That's as far as I can go with polars. Let's move to pandas.

In [8]:
df = df.to_pandas()          

Let's group by 'NumEstacao', but keeping all the info present in the rows as sets (or as a single value if all rows match)

In [16]:
def set_aggregation(x):

    if len(set(x)) == 1:
        return x.iloc[0]  
    else:
        return set(x)  

dfg = df.groupby('NumEstacao').agg(lambda x: set_aggregation(x))
dfg.head(4)

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,...,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado
NumEstacao,,,,,,,,,,,,,,,,,,,,,
19453,LIC-LIC-01,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,11030016470,19,655742007.0,"BR 364, KM 18 - COA s/n AEROPORTO SBRB",None,AC,1200401,16K0F3E,...,-9.9925,-67.804167,A,2016-03-03,2001-10-09,705,"{4d469248e6c2d201, 4d469248e6c2d1fe, 4d469248e...",2027-07-16,None,None
19461,LIC-LIC-01,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,11030016470,19,18652020.0,"BR 364, KM 18 - COE s/n AEROPORTO SBRB",None,AC,1200401,16K0F3E,...,-9.992778,-67.804444,A,2016-03-03,2001-10-09,705,"{4d469248e6c2d208, 4d469248e6c2d20c, 4d469248e...",2027-07-16,None,None
19470,LIC-LIC-01,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,11030016470,19,18652020.0,"BR 364, KM 18 - SCI s/n AEROPORTO SBRB",None,AC,1200401,16K0F3E,...,-9.992778,-67.804722,A,2016-03-03,2001-10-09,705,"{4d469248e6c2d219, 4d469248e6c2d21a, 4d469248e...",2027-07-16,None,None
64300,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,"{105802021.0, 48422023.0, 59072012.0, 97432014...","RUA FLORIANO PEIXOTO, 358",None,AC,1200401,"{100MG7W, 200KG7W, 20M0G7W, 5M00G9W, 10M0G7W, ...",...,-9.972025,-67.813556,G,2023-08-05,2000-11-27,None,"{5afddca464627, 378dd5f8bd2f226d, 5f75c76038db...",2024-07-21,None,None


In [10]:
print(f"Percentage of initial rows kept when grouping = {len(dfg)*100/len(df):.2f}%")

Percentage of initial rows kept when grouping = 16.97%


Now we have a dataframe with all the original information, but only 16% of the rows; in which each represents a single station

## Understanding the Columns

Some columns will not yield constructive information to classify the stations. We can get rid of them.

In [18]:
dfg.columns

Index(['Status.state', 'NomeEntidade', 'NumFistel', 'NumServico', 'NumAto',
       'EnderecoEstacao', 'EndComplemento', 'SiglaUf', 'CodMunicipio',
       'DesignacaoEmissao', 'Tecnologia', 'tipoTecnologia', 'meioAcesso',
       'FreqTxMHz', 'FreqRxMHz', 'Azimute', 'CodTipoClasseEstacao',
       'ClassInfraFisica', 'CompartilhamentoInfraFisica', 'CodTipoAntena',
       'CodEquipamentoAntena', 'GanhoAntena', 'FrenteCostaAntena',
       'AnguloMeiaPotenciaAntena', 'AnguloElevacao', 'Polarizacao',
       'AlturaAntena', 'CodEquipamentoTransmissor', 'PotenciaTransmissorWatts',
       'Latitude', 'Longitude', 'CodDebitoTFI', 'DataLicenciamento',
       'DataPrimeiroLicenciamento', 'NumRede', '_id', 'DataValidade',
       'NumFistelAssociado', 'NomeEntidadeAssociado'],
      dtype='object')

From [Anatel](https://sistemas.anatel.gov.br/anexar-api/publico/anexos/download/9cfc11fc83fcfc2d2586cdb887f72cb5), we can understand what each column refers to.

Status.state: Situação da solicitação na Anatel.

NumFistel: É um código numérico definido pela Anatel, composto de onze dígitos que identifica a autorização que a empresa possui para determinado serviço em determinada região.

NumServico: Código do serviço de telecomunicações na Anatel.

NumAto: Número do Ato de Autorização de Uso de Radiofrequência, sendo que os quatro últimos dígitos se referem ao ano da assinatura do respectivo ato. Exemplo XYZW2017, ato XYZW de 2017.

EnderecoEstacao: Endereço completo onde a estação está instalada.

EndComplemento: Complemento do endereço, caso haja.

DesignacaoEmissao: Designação de emissão. Para uma completa designação da emissão, necessitamos sempre de 9 caracteres alfanuméricos. Os quatros primeiros representam a _largura de faixa necessária_; os três seguintes as _características básicas_ e os dois últimos as _características adicionais facultativas_.
- Largura de faixa necessária: Para uma dada classe de emissão, o valor mínimo da largura de faixa ocupada pela emissão, suficiente para garantir a transmissão da informação com a velocidade de transmissão e com a qualidade requerida para o sistema empregado, nas condições especificadas. A largura de faixa necessária será sempre expressa por meio de três algarismos, que indicam os três primeiros algarismos significativos de largura de faixa necessária e uma letra que ocupa a posição da virgula decimal e representa a Unidade de largura de faixa e será H para Hertz, K para Kilohertz, M para Megahertz ou G para Gigahertz.


In [20]:
dfg['DesignacaoEmissao']

NumEstacao
19453                                                   16K0F3E
19461                                                   16K0F3E
19470                                                   16K0F3E
64300         {100MG7W, 200KG7W, 20M0G7W, 5M00G9W, 10M0G7W, ...
356271                                                  16K0F3E
                                    ...                        
1015180580                                              40M0D7W
1015180598                                              40M0D7W
1015180601                                              29M6D7W
1015180610                                              29M6D7W
1015180628                                              56M0D7W
Name: DesignacaoEmissao, Length: 1500, dtype: object

In [19]:
dfg = dfg.drop(['Status.state', 'NumFistel'], axis=1)
